In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import dask.dataframe as dd
from toolbox.data_aggregation import *
from toolbox.linear_reg import *
import datetime
from datetime import timedelta, date
import numpy as np

# Data Preprocessing

## Train data
### Unit sales and promotion by store and date

In [2]:
df_train = pd.read_csv(
    'data/train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    # comment the following line for full dataset processing
    skiprows=range(1, 115688780)  # 2017-01-01 sharp = 101688780! (115688780 for test)
)

df_test = pd.read_csv(
    "data/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [3]:
df_unit_sales = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_unit_sales.columns = df_unit_sales.columns.get_level_values(1)

In [4]:
df_unit_sales.head()

date                2017-05-15  2017-05-16  2017-05-17  2017-05-18  \
store_nbr item_nbr                                                   
1         96995            0.0    0.000000    0.000000    0.000000   
          99197            0.0    1.098612    1.098612    0.693147   
          103520           0.0    1.609438    0.693147    0.000000   
          103665           0.0    1.791759    0.000000    0.693147   
          105574           0.0    2.197225    2.302585    2.302585   

date                2017-05-19  2017-05-20  2017-05-21  2017-05-22  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000         0.0    0.000000   
          99197       1.098612    0.000000         0.0    0.000000   
          103520      0.000000    1.098612         0.0    0.693147   
          103665      2.197225    1.386294         0.0    1.098612   
          105574      1.945910    0.693147         0.0    1.945910   

date                2017-05-23  2017-05-24     ...      2017-08-06  \
store_nbr item_nbr                             ...                   
1         96995       0.000000    0.000000     ...        1.098612   
          99197       0.000000    1.098612     ...        0.000000   
          103520      1.098612    1.609438     ...        0.000000   
          103665      1.791759    0.000000     ...        0.693147   
          105574      2.772589    1.791759     ...        0.000000   

date                2017-08-07  2017-08-08  2017-08-09  2017-08-10  \
store_nbr item_nbr                                                   
1         96995       1.098612    0.000000    0.000000    0.693147   
          99197       1.098612    0.000000    1.098612    0.000000   
          103520      0.000000    1.386294    0.000000    1.386294   
          103665      1.098612    0.000000    2.079442    2.302585   
          105574      1.791759    2.079442    1.945910    2.397895   

date                2017-08-11  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                              
1         96995       0.000000    0.000000    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    0.000000    0.000000    0.000000  
          103520      0.693147    0.693147    0.693147    0.000000    0.000000  
          103665      1.098612    0.000000    0.000000    0.693147    0.693147  
          105574      1.791759    1.791759    0.000000    1.386294    1.609438  

[5 rows x 93 columns]

In [5]:
df_promo_train = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
df_promo_train.columns = df_promo_train.columns.get_level_values(1)

df_promo_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
df_promo_test.columns = df_promo_test.columns.get_level_values(1)
df_promo_test = df_promo_test.reindex(df_promo_train.index).fillna(False)

df_promo = pd.concat([df_promo_train, df_promo_test], axis=1)
del df_promo_train, df_promo_test

In [6]:
df_promo.head()

date                2017-05-15  2017-05-16  2017-05-17  2017-05-18  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-05-19  2017-05-20  2017-05-21  2017-05-22  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-05-23  2017-05-24     ...      2017-08-22  \
store_nbr item_nbr                             ...                   
1         96995          False       False     ...           False   
          99197          False       False     ...           False   
          103520         False       False     ...           False   
          103665         False       False     ...           False   
          105574         False       False     ...           False   

date                2017-08-23  2017-08-24  2017-08-25  2017-08-26  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-27  2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                              
1         96995          False       False       False       False       False  
          99197          False       False       False       False       False  
          103520         False       False       False       False       False  
          103665         False       False       False       False       False  
          105574         False       False       False       False       False  

[5 rows x 109 columns]

In [7]:
del df_train

### Item information

In [8]:
df_items = pd.read_csv(
    "data/items.csv",
).set_index("item_nbr")
df_items = df_items.reindex(df_unit_sales.index.get_level_values(1))
df_items.head()

,family,class,perishable
item_nbr,,,
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1
105574,GROCERY I,1045,0


Feature to use: class & perishable. Class is already numeric coded, which should perform well in tree models. For other models, binary encoding can be used instead.

### Store information

In [9]:
df_stores = pd.read_csv("data/stores.csv").set_index("store_nbr")
df_stores_original = df_stores.copy()
df_stores_expanded = df_stores.reindex(df_unit_sales.index.get_level_values(0))

In [10]:
df_stores_expanded.head()

,city,state,type,cluster
store_nbr,,,,
1,Quito,Pichincha,D,13
1,Quito,Pichincha,D,13
1,Quito,Pichincha,D,13
1,Quito,Pichincha,D,13
1,Quito,Pichincha,D,13


In [11]:
df_stores_original['state'].value_counts()

Pichincha                         19
Guayas                            11
Santo Domingo de los Tsachilas     3
Azuay                              3
Manabi                             3
Los Rios                           2
Tungurahua                         2
Cotopaxi                           2
El Oro                             2
Chimborazo                         1
Santa Elena                        1
Pastaza                            1
Imbabura                           1
Esmeraldas                         1
Bolivar                            1
Loja                               1
Name: state, dtype: int64

In [12]:
df_stores_original['city'].value_counts()

Quito            18
Guayaquil         8
Cuenca            3
Santo Domingo     3
Manta             2
Machala           2
Latacunga         2
Ambato            2
Cayambe           1
Salinas           1
Loja              1
Daule             1
Babahoyo          1
Esmeraldas        1
El Carmen         1
Playas            1
Riobamba          1
Puyo              1
Libertad          1
Quevedo           1
Ibarra            1
Guaranda          1
Name: city, dtype: int64

Choose to use type and cluster. Type can be numeric coded.

In [13]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
encoder = LabelEncoder()
encoder.fit(df_stores_expanded['type'])
df_stores_expanded['type'] = encoder.transform(df_stores_expanded['type'])

### Oil price by date

In [14]:
df_oil = pd.read_csv("data/oil.csv", parse_dates = ['date']).set_index('date')
df_oil = df_oil.fillna(method='backfill')['dcoilwtico']

### Holiday information

In [15]:
df_holidays = pd.read_csv("data/holidays_events.csv")
df_holidays['date'] = df_holidays['date'].astype(np.datetime64)
df_holidays.head()

C:\Users\xiaoyu\.edm\envs\mle3\lib\site-packages\pandas\core\internals.py:3224: FutureWarning: Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.
  return self.apply('astype', dtype=dtype, **kwargs)


,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [16]:
df_holidays["locale"].value_counts()

National    174
Local       152
Regional     24
Name: locale, dtype: int64

In [17]:
df_holidays["type"].value_counts()

Holiday       221
Event          56
Additional     51
Transfer       12
Bridge          5
Work Day        5
Name: type, dtype: int64

In [18]:
len(df_holidays)

350

### Holiday inforamtion using the same index and column with unit sales and promotion

In [19]:
df_holidays_expanded = pd.DataFrame(np.zeros(df_promo.values.shape), index=df_promo.index.copy(), columns=df_promo.columns.copy())

### Populate the table

In [20]:
for i in df_holidays.index.values[:]:
    # Remove non-holidays
    if df_holidays.loc[i, 'transferred'] == True or df_holidays.loc[i, 'type'] == 'Work Day':
        print(df_holidays.loc[i, 'date'], "is not a holiday")
    else:
        # Populate table according to holiday locale
        if df_holidays.loc[i, 'locale'] == 'National':
            print(df_holidays.loc[i, 'date'], "is a National holiday")
            df_holidays_expanded.iloc[:, df_holidays_expanded.columns.get_level_values(0) == df_holidays.loc[i, 'date']] = 1
        elif df_holidays.loc[i, 'locale'] == 'Regional':
            print(df_holidays.loc[i, 'date'], "is a Regional holiday to ", df_holidays.loc[i, 'locale_name'])
            df_holidays_expanded.iloc[
                df_holidays_expanded.index.get_level_values(0).isin(
                    df_stores.loc[df_stores_original['state'] == df_holidays.loc[i, 'locale_name']].index.values
                ),
                df_holidays_expanded.columns.get_level_values(0) == df_holidays.loc[i, 'date']
            ] = 1
        elif df_holidays.loc[i, 'locale'] == 'Local':
            print(df_holidays.loc[i, 'date'], "is a Local holiday to ", df_holidays.loc[i, 'locale_name'])
            df_holidays_expanded.iloc[
                df_holidays_expanded.index.get_level_values(0).isin(
                    df_stores.loc[df_stores_original['city'] == df_holidays.loc[i, 'locale_name']].index.values
                ),
                df_holidays_expanded.columns.get_level_values(0) == df_holidays.loc[i, 'date']
            ] = 1
        else:
            print("Warning!")

2012-03-02 00:00:00 is a Local holiday to  Manta
2012-04-01 00:00:00 is a Regional holiday to  Cotopaxi
2012-04-12 00:00:00 is a Local holiday to  Cuenca
2012-04-14 00:00:00 is a Local holiday to  Libertad
2012-04-21 00:00:00 is a Local holiday to  Riobamba
2012-05-12 00:00:00 is a Local holiday to  Puyo
2012-06-23 00:00:00 is a Local holiday to  Guaranda
2012-06-25 00:00:00 is a Regional holiday to  Imbabura
2012-06-25 00:00:00 is a Local holiday to  Latacunga
2012-06-25 00:00:00 is a Local holiday to  Machala
2012-07-03 00:00:00 is a Local holiday to  Santo Domingo
2012-07-03 00:00:00 is a Local holiday to  El Carmen
2012-07-23 00:00:00 is a Local holiday to  Cayambe
2012-08-05 00:00:00 is a Local holiday to  Esmeraldas
2012-08-10 00:00:00 is a National holiday
2012-08-15 00:00:00 is a Local holiday to  Riobamba
2012-08-24 00:00:00 is a Local holiday to  Ambato
2012-09-28 00:00:00 is a Local holiday to  Ibarra
2012-10-07 00:00:00 is a Local holiday to  Quevedo
2012-10-09 00:00:00 is 

# Generate single date samples

In [21]:
def get_timespan(df, dt, minus, periods):
    return df[
        pd.date_range(dt - timedelta(days=minus), periods=periods)
    ]

In [22]:
def prepare_sample(sales_raw, promo_raw, oil_raw, holiday_raw, store_raw, sample_dt, is_train=True):
    temp_dict = {}
    for i in [3, 7, 14, 28]:
        temp_dict["mean_{}".format(i)] = get_timespan(sales_raw, sample_dt, i, i).mean(axis=1).values
        temp_dict["promo_{}_sum".format(i)] = get_timespan(promo_raw, sample_dt, i, i).sum(axis=1).values
        temp_dict["oil_{}_mean".format(i)] = get_timespan(oil_raw, sample_dt, i, i).mean()
        temp_dict["holiday_{}_sum".format(i)] = get_timespan(holiday_raw, sample_dt, i, i).sum(axis=1).values
    X = pd.DataFrame(temp_dict)
    for i in range(16):
        X["promo_{}".format(i)] = promo_raw[
            sample_dt + timedelta(days=i)].values.astype(np.uint8)
    X['store_type'] = store_raw['type'].values
    X['store_cluster'] = store_raw['cluster'].values
    X['item_perishable'] = df_items['perishable'].values
    X['item_class'] = df_items['class'].values
    if is_train:
        y = sales_raw[
            pd.date_range(sample_dt, periods=16)
        ].values
        return X, y
    return X

# Generate dataset

In [23]:
date(2017, 7, 29) - date(2017, 2, 1)

datetime.timedelta(178)

In [24]:
some_date = date(2017, 7, 1)
X_l, y_l = [], []
for i in range(4):
    delta = timedelta(days=i*7)
    X_tmp, y_tmp = prepare_sample(
        df_unit_sales, df_promo, df_oil, df_holidays_expanded, df_stores_expanded,
        some_date + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_sample(
    df_unit_sales, df_promo, df_oil, df_holidays_expanded, df_stores_expanded, date(2017, 7, 29)
)
X_test = prepare_sample(
    df_unit_sales, df_promo, df_oil, df_holidays_expanded, df_stores_expanded, date(2017, 8, 16), is_train=False
)

In [25]:
X_train.values.shape

(633640, 36)

In [26]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 633640 entries, 0 to 158409
Data columns (total 36 columns):
holiday_14_sum     633640 non-null float64
holiday_28_sum     633640 non-null float64
holiday_3_sum      633640 non-null float64
holiday_7_sum      633640 non-null float64
mean_14            633640 non-null float64
mean_28            633640 non-null float64
mean_3             633640 non-null float64
mean_7             633640 non-null float64
oil_14_mean        633640 non-null float64
oil_28_mean        633640 non-null float64
oil_3_mean         633640 non-null float64
oil_7_mean         633640 non-null float64
promo_14_sum       633640 non-null int64
promo_28_sum       633640 non-null int64
promo_3_sum        633640 non-null int64
promo_7_sum        633640 non-null int64
promo_0            633640 non-null uint8
promo_1            633640 non-null uint8
promo_2            633640 non-null uint8
promo_3            633640 non-null uint8
promo_4            633640 non-null uint8
prom

# Save to file

In [27]:
X_train.to_csv('data/X_train_2017.csv', index=False)

In [28]:
np.savetxt("data/y_train_2017.csv", y_train, delimiter=",")

In [29]:
X_val.to_csv('data/X_val_2017.csv', index=False)

In [30]:
np.savetxt("data/y_val_2017.csv", y_val, delimiter=",")

In [31]:
X_test.to_csv('data/X_test.csv', index=False)